In [4]:
import os
import cv2
import glob
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist

CROP_SIZE      = 412
CROP_SIZE_HALF = CROP_SIZE / 2

folders = ["task_occlusion_data_01-2020_11_16_04_38_33-yolo 1.1",
           "task_occlusion_data_02-2020_11_17_06_31_46-yolo 1.1",
           "task_occlusion_data_03-2020_11_18_05_41_15-yolo 1.1",
           "task_football image training-2020_11_30_12_49_12-yolo 1.1",
           "task_project-2020_11_28_08_17_44-yolo 1.1"]

folders

['task_occlusion_data_01-2020_11_16_04_38_33-yolo 1.1',
 'task_occlusion_data_02-2020_11_17_06_31_46-yolo 1.1',
 'task_occlusion_data_03-2020_11_18_05_41_15-yolo 1.1',
 'task_football image training-2020_11_30_12_49_12-yolo 1.1',
 'task_project-2020_11_28_08_17_44-yolo 1.1']

In [27]:
def get_files():
    images = sorted(glob.glob("*.jpg"))
    texts  = sorted(glob.glob("*.txt"))
    
    return images, texts

def read_files(image_fname, text_fname):
    image = cv2.imread(image_fname, cv2.IMREAD_COLOR)
    text  = pd.read_csv(text_fname, delimiter=" ", header=None, names=HEADER)
    text  = text.query('label in [0, 1]').reset_index(drop=True)
    
    return image, text

def get_image_shape(image):
    width  = image.shape[1]
    height = image.shape[0]
    
    return width, height

def convert_yolo_to_pixel_coord(yolo_coord, image_width, image_height):
    pixel_coord = pd.DataFrame()
    pixel_coord['label'] = yolo_coord['label']
    pixel_coord['x']     = yolo_coord['x'] * image_width
    pixel_coord['y']     = yolo_coord['y'] * image_height
    pixel_coord['w']     = yolo_coord['w'] * image_width
    pixel_coord['h']     = yolo_coord['h'] * image_height
    pixel_coord = pixel_coord.applymap(int)
    
    return pixel_coord

def get_center_coord(pixel_coord):
    return pixel_coord[['x', 'y']]

def find_neighbor(center_coord, crop_size):
    cross_dist_mat = pd.DataFrame(cdist(center_coord, center_coord))
    near_dist_mat  = cross_dist_mat < (crop_size / 2) + 100
    
    return near_dist_mat

def crop_boxwise_image(image, coord, CROP_SIZE):
    crop_image_center_x = coord.x
    crop_image_center_y = coord.y

    crop_image_start_x  = int(crop_image_center_x - CROP_SIZE_HALF)
    crop_image_end_x    = int(crop_image_center_x + CROP_SIZE_HALF)
    crop_image_start_y  = int(crop_image_center_y - CROP_SIZE_HALF)
    crop_image_end_y    = int(crop_image_center_y + CROP_SIZE_HALF)

    crop_image = image[crop_image_start_y:crop_image_end_y,
                       crop_image_start_x:crop_image_end_x]
    
    crop_image = pad_crop_image(crop_image, CROP_SIZE)
    
    return crop_image, crop_image_start_x, crop_image_start_y

def pad_crop_image(crop_image, CROP_SIZE):
    crop_image_shape = crop_image.shape

    if crop_image.shape[0] < (CROP_SIZE):
        crop_image = np.pad(crop_image, ((0, CROP_SIZE - crop_image_shape[0]), (0, 0), (0, 0)), 'constant', constant_values=(0))

    if crop_image.shape[1] < (CROP_SIZE):
        crop_image = np.pad(crop_image, ((0, 0), (0, CROP_SIZE - crop_image_shape[1]), (0, 0)), 'constant', constant_values=(0))
    
    return crop_image

def write_crop_image(RESULTS_PATH, crop_image, train_text, ct):
    image_output_fname = "frame_{:06d}.jpg".format(ct)
    cv2.imwrite(os.path.join(RESULTS_PATH, image_output_fname), crop_image)
    train_text.append("data/obj_train_data/" + image_output_fname)
    
    return train_text

def write_yolo_coords(RESULTS_PATH, yolo_coord_results, ct):
    yolo_coord_output_fname = "frame_{:06d}.txt".format(ct)
    yolo_coord_results = pd.DataFrame(yolo_coord_results)
    yolo_coord_results.to_csv(os.path.join(RESULTS_PATH, yolo_coord_output_fname), header=False, index=False, sep=' ')
    

In [37]:
HEADER = ["label", "x", "y", "w", "h"]
ROOT_PATH    = "/Users/jonghyun/Desktop/training_data"
TASK_PATH    = os.path.join(ROOT_PATH, "task_merged")
RESULTS_PATH = os.path.join(TASK_PATH, "obj_train_data")

ct = 0
train_text = []
for folder_num, folder in enumerate(folders[:1]):
    print(f"folder number: {folder_num + 1} / total folder number: {len(folders)}")
    os.chdir(os.path.join(ROOT_PATH, folder, "obj_train_data"))
    
    images, texts = get_files()
    
    for i in tqdm(range(len(images))):
        image_fname = images[i]
        text_fname  = texts[i]

        image, yolo_coord = read_files(image_fname, text_fname)
        IMAGE_WIDTH, IMAGE_HEIGHT = get_image_shape(image)

        pixel_coord   = convert_yolo_to_pixel_coord(yolo_coord, IMAGE_WIDTH, IMAGE_HEIGHT)
        center_coord  = get_center_coord(pixel_coord)
        near_dist_mat = find_neighbor(center_coord, CROP_SIZE) 
        
        for j, coord in pixel_coord.iterrows():
            crop_image, crop_image_start_x, crop_image_start_y = crop_boxwise_image(image, coord, CROP_SIZE)
            neighbor_box_index = near_dist_mat[near_dist_mat.iloc[j]].index.to_numpy()

            yolo_coord_results = []
            for k in neighbor_box_index:
                box_label    = pixel_coord.iloc[k].label
                box_center_x = round((pixel_coord.iloc[k].x - crop_image_start_x) / CROP_SIZE, 6)
                box_center_y = round((pixel_coord.iloc[k].y - crop_image_start_y) / CROP_SIZE, 6)
                box_w        = round(pixel_coord.iloc[k].w / CROP_SIZE, 6)
                box_h        = round(pixel_coord.iloc[k].h / CROP_SIZE, 6)

                if (box_center_x > 0.1 and box_center_x < 0.9) and (box_center_y > 0.1 and box_center_y < 0.9):
                    yolo_coord_results.append([box_label, box_center_x, box_center_y, box_w, box_h])
            
            if yolo_coord_results:
                train_text = write_crop_image(RESULTS_PATH, crop_image, train_text, ct)
                write_yolo_coords(RESULTS_PATH, yolo_coord_results, ct)
                ct += 1
            else:
                pass
                    
train_text = pd.DataFrame(train_text)
train_text.to_csv(os.path.join(TASK_PATH, 'train.txt'), header=False, index=False)

folder number: 0 / total folder number: 5


KeyboardInterrupt: 